# Combat Simulator Comparison v2

Setup-aware comparison of `sc2helper` (ground truth) vs numpy Lanchester simulator.

In [1]:
from __future__ import annotations

import lzma
import pickle
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score

ROOT = Path.cwd()
if not (ROOT / "phantom").exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from phantom.mock.combat_sim import generate_mock_combat_dataset
from phantom.mock.hp_ratio_sim import predict_outcome as hp_ratio_predict_outcome

DATASET_PATH = ROOT / "resources/combat_sim/mock.pkl.xz"
RUN_DIRECT = True
SIMULATION_COUNT = 3000
SPAWN_COUNT = 10
SEED = 1337


In [2]:
if RUN_DIRECT or not DATASET_PATH.exists():
    records = generate_mock_combat_dataset(
        simulation_count=SIMULATION_COUNT,
        spawn_count=SPAWN_COUNT,
        use_position=True,
        seed=SEED,
    )
    source = "direct"
else:
    with lzma.open(DATASET_PATH, "rb") as f:
        records = pickle.load(f)
    source = str(DATASET_PATH)

df = pd.DataFrame(records)
if "pred_outcome_hp_ratio" not in df.columns:
    def _hp_ratio_from_units(units: list[dict]) -> float:
        units1 = [u for u in units if not bool(u["is_enemy"])]
        units2 = [u for u in units if bool(u["is_enemy"])]
        return hp_ratio_predict_outcome(units1, units2)

    df["pred_outcome_hp_ratio"] = df["units"].map(_hp_ratio_from_units)
required = {"setup", "parameter_name", "parameter_value", "true_outcome", "pred_outcome"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"dataset missing setup-comparison columns: {sorted(missing)}")

df = df.sort_values(["setup", "parameter_value"]).reset_index(drop=True)
print(f"Loaded {len(df):,} rows from {source}")
df.head(3)


Combat simulations (mock setups):   1%|          | 27/3000 [00:00<00:11, 266.47it/s]

-0.11651126207887036 -0.1149673481439378
-0.1123169000517872 -0.036078462271950204
0.2379489478851378 0.1473237536876997
0.9697116027780018 -0.6379733598421216
0.8458964357123869 0.7309925177767586
-0.6124283743916976 -0.27652047954580866
-0.19419009455377362 -0.2500624964376498
0.08851438199397943 0.23099774290220185
0.9987799284316481 0.9995772433612267
-0.6430108602708567 -0.3817019193816571
0.25731917204333415 0.3460968263150917
-0.42033307697877886 -0.3576730555311111
-0.055833784870211185 -0.027114492038155427
-0.08926179075879347 -0.03816107178074367
-0.41490280875676855 -0.36302523076251847
-0.08585543574301917 -0.09047685554638277
-0.5836513169779564 -0.29067646522956264
-0.04192459732993963 0.002311423407956378
0.12195011396532254 -0.0012288821472593442
0.15870907813601387 0.25491667054846445
-0.5839896622574741 -0.2191914067071312
-0.07586783269821373 -0.24521424732294653
-0.17672995625785537 -0.20202540750493198
0.19605016347858892 0.2328303628118314
-0.12444877102649884 -0

Combat simulations (mock setups):   3%|▎         | 84/3000 [00:00<00:11, 256.68it/s]

-0.08926179075879347 -0.08855066479689332
-0.8378307870561432 -0.8071037662951943
-0.22014402454433904 -0.2507822209322687
-0.24197178996305554 -0.23713850561459449
-0.3548445459476805 -0.29250162007452485
0.16425690075039553 0.23058285894505465
0.08519346992821335 0.1273881734358866
-0.16339786796330458 -0.0601427029013005
-0.588963967653016 -0.22605226773068618
-0.3518282674625852 -0.2602697008713623
-0.09259314853521963 -0.11346022370575226
0.05644484275527038 0.10526807423187035
0.38816534574866157 0.3091911163338046
-0.13577230067184734 -0.22034497157904276
-0.6871554820139847 -0.35462290316255146
0.11415320977252709 0.06253341209568807
-0.08926179075879347 -0.08827862290037637
0.11395695152058838 0.33502550324735947
-0.08926179075879347 -0.2043605541470726
-0.09324415831602345 -0.12776120164158705
-0.1492428135724665 -0.22616284779596965
0.1262192382968335 0.30458293844773754
0.2267464741708532 0.2763181128352898
-0.5081242639319751 -0.42310631947250865
-0.14748935485560988 -0.21

Combat simulations (mock setups):   5%|▍         | 137/3000 [00:00<00:11, 256.65it/s]

...

Combat simulations (mock setups):  98%|█████████▊| 2939/3000 [00:12<00:00, 247.46it/s]

-0.3038367748639304 -0.2342214972060825
-0.398742905594408 -0.3202681336190559
-0.6059476670024291 -0.40014941952951016
-0.35972806652532996 -0.34009136329330736
-0.3756009377988201 -0.25397439002646466
0.10125709183211451 0.2062972393050903
0.3393399396785649 0.49835975124535353
0.5635301068898477 0.6373686397367723
0.757847252739713 0.852577789590801
-0.14067884050625573 -0.17713364163942058
-0.9434753727219871 -0.6520910161406538
-0.524524561826406 -0.5976488548033607
-0.8400691542500024 -0.6057509861719473
-0.28368257584201895 -0.20366626724990086
-0.21746194536586327 -0.25971426797154884
0.9403035905522266 0.913276465867504
-0.046417382876874774 -0.06059354875880038
-0.10547810905217109 -0.2036689503295038
-0.517984742728627 -0.5644448861016627
-0.043173905729431854 -0.024708341295171896
0.49637969484114886 0.2433400627613623
-0.07242000097899423 -0.05412846423312251
-0.0864146586976214 -0.10382825430065054
-0.1651166086578893 -0.2695640854469245
-0.13312229849358523 -0.1109688145

Combat simulations (mock setups): 100%|██████████| 3000/3000 [00:12<00:00, 243.85it/s]

0.005669843453005358 0.11378740868808195
-0.04979888217577172 -0.08150621182944363
0.5406553999364591 0.49749845071298693
0.2754581330902004 0.23988769206992155
0.6675202571853661 0.47207263149091067
-0.178905256897312 -0.16915045665680087
-0.08926179075879347 -0.08724471633628555
-0.0957802795366051 -0.23730879197667357
-0.246042704781692 -0.29819105099755683
0.44167832823091807 0.5117994544663981
-0.24097679840203823 -0.26216270196576613
0.1602269981912889 0.20531870486105816
0.23593746274297064 0.3869495475090123
0.2448961428116637 0.20358556581038081
-0.43971786428465875 -0.0453730922078968
-0.505113876235788 -0.2189410828303828
0.3382561088540321 0.4315022433703776
-0.3645867197781782 -0.2830431412770509
0.6285150601750692 0.361204517359531
0.1648763955382918 0.21091355525472705
-0.09230513735664486 -0.0933335556737313
-0.1600866189607129 -0.1625730204783834
-0.6393667983746071 -0.4807703585297542
-0.14827266625676452 -0.22750176782342976
-0.036154679623696326 -0.09475517889587312

,setup,parameter_name,parameter_value,units,true_outcome,pred_outcome,pred_outcome_hp_ratio,true_advantage_log,true_bitterness_log
0,circle,radius,2.0,"[{'tag': 1, 'is_enemy': False, 'is_flying': Fa...",-0.386991,-0.357673,-0.020223,-0.244688,-0.244688
1,circle,radius,2.0,"[{'tag': 1, 'is_enemy': False, 'is_flying': Fa...",-0.398734,-0.019757,0.051829,-0.254359,-0.254359
2,circle,radius,2.0,"[{'tag': 1, 'is_enemy': False, 'is_flying': Fa...",0.149644,0.239086,0.031325,0.081050,-0.081050


In [3]:
setups = sorted(df["setup"].unique())
fig = make_subplots(rows=1, cols=len(setups), subplot_titles=setups)

for col, setup in enumerate(setups, start=1):
    part = df[df["setup"] == setup]
    fig.add_trace(
        go.Scatter(
            x=part["true_outcome"],
            y=part["pred_outcome"],
            mode="markers",
            marker={
                "size": 6,
                "opacity": 0.45,
                "color": part["parameter_value"],
                "colorscale": "Viridis",
                "showscale": col == len(setups),
                "colorbar": {"title": "parameter"},
            },
            name=setup,
            showlegend=False,
        ),
        row=1,
        col=col,
    )
    fig.add_trace(
        go.Scatter(x=[-1, 1], y=[-1, 1], mode="lines", line={"dash": "dash", "color": "black"}, showlegend=False),
        row=1,
        col=col,
    )
    fig.update_xaxes(title_text="true (sc2helper)", row=1, col=col, range=[-1, 1])
    fig.update_yaxes(title_text="pred (numpy)", row=1, col=col, range=[-1, 1])

fig.update_layout(height=440, width=max(1100, 320 * len(setups)), template="plotly_white")
fig.show()


In [4]:
rows = []
for (setup, parameter_value), part in df.groupby(["setup", "parameter_value"], sort=True):
    if len(part) < 2:
        pred_numpy_score = np.nan
        pred_hp_ratio_score = np.nan
    else:
        pred_numpy_score = r2_score(part["true_outcome"], part["pred_outcome"])
        pred_hp_ratio_score = r2_score(part["true_outcome"], part["pred_outcome_hp_ratio"])
    rows.append(
        {
            "setup": setup,
            "parameter_value": float(parameter_value),
            "score": 1.0,
            "source": "true",
        }
    )
    rows.append(
        {
            "setup": setup,
            "parameter_value": float(parameter_value),
            "score": float(pred_numpy_score),
            "source": "pred_numpy",
        }
    )
    rows.append(
        {
            "setup": setup,
            "parameter_value": float(parameter_value),
            "score": float(pred_hp_ratio_score),
            "source": "pred_hp_ratio",
        }
    )

score_df = pd.DataFrame(rows)
score_df.head(9)


,setup,parameter_value,score,source
0,circle,2.0,1.000000,true
1,circle,2.0,0.145132,pred_numpy
2,circle,2.0,-0.134986,pred_hp_ratio
3,circle,4.0,1.000000,true
4,circle,4.0,0.164786,pred_numpy
5,circle,4.0,-0.064115,pred_hp_ratio
6,circle,6.0,1.000000,true
7,circle,6.0,0.102450,pred_numpy
8,circle,6.0,-0.164334,pred_hp_ratio


In [5]:
fig = make_subplots(rows=1, cols=len(setups), subplot_titles=setups)
for col, setup in enumerate(setups, start=1):
    part = score_df[score_df["setup"] == setup].sort_values("parameter_value")
    for source, color in [("true", "#2ca02c"), ("pred_numpy", "#1f77b4"), ("pred_hp_ratio", "#ff7f0e")]:
        sub = part[part["source"] == source]
        fig.add_trace(
            go.Scatter(
                x=sub["parameter_value"],
                y=sub["score"],
                mode="lines+markers",
                name=source,
                line={"color": color},
                showlegend=(col == 1),
            ),
            row=1,
            col=col,
        )
    fig.update_xaxes(title_text="parameter", row=1, col=col)
    fig.update_yaxes(title_text="r2", row=1, col=col)

fig.update_layout(height=420, width=max(1100, 320 * len(setups)), template="plotly_white")
fig.show()
